# [STARTER] Udaplay Project

## Part 02 - Agent

In this part of the project, you'll use your VectorDB to be part of your Agent as a tool.

You're building UdaPlay, an AI Research Agent for the video game industry. The agent will:
1. Answer questions using internal knowledge (RAG)
2. Search the web when needed
3. Maintain conversation state
4. Return structured outputs
5. Store useful information for future use

### Setup

In [1]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [ ]:
# Import the necessary libs
import os
import json

from lib.agents import Agent
from lib.llm import LLM
from lib.state_machine import Run
from lib.messages import BaseMessage
from lib.tooling import tool
from lib.rag import RAG

from typing import List,Dict
from datetime import datetime

import chromadb
from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction
from dotenv import load_dotenv
from tavily import TavilyClient

In [3]:
# Load environment variables
load_dotenv()

True

In [4]:
api_key = os.getenv("TAVILY_API_KEY")
client_tavily = TavilyClient(api_key=api_key)

In [5]:
result = client_tavily.search("What's Nintendo?")

In [6]:
result

{'query': "What's Nintendo?",
 'follow_up_questions': None,
 'answer': None,
 'images': [],
 'results': [{'url': 'https://en.wikipedia.org/wiki/Nintendo',
   'title': 'Nintendo - Wikipedia',
   'content': '**Nintendo Co., Ltd.** is a Japanese multinational video game and entertainment company headquartered in Kyoto. The history of Nintendo began when craftsman Fusajiro Yamauchi founded the company to produce handmade *hanafuda* playing cards. The company became internationally dominant in the 1980s after the arcade release of *Donkey Kong "Donkey Kong (1981 video game)")* (1981) and the Nintendo Entertainment System, which launched outside of Japan alongside *Super Mario Bros.* in 1985. Nintendo was founded as Nintendo Koppai on 23 September 1889 by craftsman Fusajiro Yamauchi in Shimogyō-ku, Kyoto, Japan, as an unincorporated establishment, to produce and distribute Japanese playing cards, or karuta (かるた; from Portuguese *carta*, \'card\'), most notably *hanafuda* (花札, \'flower cards\

In [ ]:
# Reuse the same embedding function
embedding_fn = OpenAIEmbeddingFunction()

# Connect  to the existing vectorDB
client = chromadb.PersistentClient(path="chromadb_data")

In [8]:
# Load existing collection
collection = client.get_collection(name="udaplay51", embedding_function=embedding_fn)

In [9]:
# Validating that Im able to pull data from the collection created in Udaplay_01_starter_project.ipynb
collection.peek(1)

{'ids': ['001'],
 'embeddings': array([[-0.00270652, -0.01438423, -0.0094357 , ..., -0.02086851,
         -0.00733747, -0.03789448]], shape=(1, 1536)),
 'documents': ['[PlayStation 1] Gran Turismo (1997) - A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.'],
 'uris': None,
 'included': ['metadatas', 'documents', 'embeddings'],
 'data': None,
 'metadatas': [{'Publisher': 'Sony Computer Entertainment',
   'Description': 'A realistic racing simulator featuring a wide array of cars and tracks, setting a new standard for the genre.',
   'Platform': 'PlayStation 1',
   'Name': 'Gran Turismo',
   'Genre': 'Racing',
   'YearOfRelease': 1997}]}

In [10]:
# Setting up RAG based on vectorDB
rag_llm = LLM(
    model="gpt-4o-mini",
    temperature=0.3,
)

games_rag = RAG(
    llm=rag_llm,
    vector_store = collection
)


In [11]:
# Testing RAG
result:Run = games_rag.invoke("How many games deal with racing cars?")
print(result.get_final_state()["answer"])

[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
There are two games mentioned that deal with racing cars: "Gran Turismo" (1997) and "Gran Turismo 5" (2010). Additionally, "Mario Kart 8 Deluxe" (2017) can also be considered a racing game. Therefore, there are three games that deal with racing cars.


### Tools

Build at least 3 tools:
- retrieve_game: To search the vector DB
- evaluate_retrieval: To assess the retrieval performance
- game_web_search: If no good, search the web


#### Retrieve Game Tool

In [12]:
# Create retrieve_game tool
@tool
def retrieve_game(query):
    """
    Semantic search: Finds most results in the vector DB
    args:
    - query: a question about game industry. 

    You'll receive results as list. Each element contains:
    - Platform: like Game Boy, Playstation 5, Xbox 360...)
    - Name: Name of the Game
    - YearOfRelease: Year when that game was released for that platform
    - Description: Additional details about the game
    """
    result:Run = games_rag.invoke(query)
    return result.get_final_state()["answer"]



#### Evaluate Retrieval Tool

In [ ]:
# Supportive tool to evaluate retrieval results
def evaluate_with_llm(query, retrieved_docs):
    # join all incoming documents into a single string
    docs_text = "\n".join([f"- {doc['document']}" for doc in retrieved_docs])
    
    prompt = f"""
    You are an assistant that evaluates retrieved documents for a user query so the user can determine if they are useful or not.
    It is important to notice that this feedback will be used to decide wether to retrieve more information from the web or not.
    Query: {query}
    Documents: {docs_text}
    Question: Does the documents provide an satisfactory answer to the query? Respond with:
    - 'useful': True or False
    - 'description': short explanation
    Return your response as JSON.
    """

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}]
    )

    result = json.loads(response.choices[0].message.content)
    return result

In [ ]:
# Create evaluate_retrieval tool
@tool
def evaluate_retrieval(query: str, retrieved_docs: list):
    """
    Based on the user's question and on the list of retrieved documents, 
    it will analyze the usability of the documents to respond to that question. 
    args: 
    - question: original question from user
    - retrieved_docs: retrieved documents most similar to the user query in the Vector Database
    The result includes:
    - useful: whether the documents are useful to answer the question
    - description: description about the evaluation result
    """
    return evaluate_with_llm(query, retrieved_docs)

#### Game Web Search Tool

In [ ]:
# Create game_web_search tool
# Please use Tavily client to search the web
@tool
def game_web_search(query: str, search_depth: str = "basic") -> Dict:
    """
    Semantic search: Finds most results in the vector DB
    args:
    - question: a question about game industry. 
    """
    # Perform the search
    search_result = client_tavily.search(
        query=query,
        search_depth=search_depth,
        include_answer=True,
        include_raw_content=False,
        include_images=False
    )
    
    # Format the results
    formatted_results = {
        "answer": search_result.get("answer", ""),
        "results": search_result.get("results", []),
        "search_metadata": {
            "timestamp": datetime.now().isoformat(),
            "query": query
        }
    }
    
    return formatted_results

### Agent

In [ ]:
# Create your Agent abstraction using StateMachine
# Equip with an appropriate model
# Craft a good set of instructions 
# Plug all Tools you developed
# Reduced the tempature to 0.1 to make it more deterministic and make sure it use tools wisely
agentic_rag = Agent(
    model_name="gpt-4o-mini",
    temperature=0.1,
    tools=[retrieve_game, evaluate_retrieval, game_web_search],
    instructions=(
        "You are an Agentic RAG assistant that can intelligently decide which tools to use "
        "to answer user questions. Reason about about the response, change the query and call the tool again if needed "
        "in order to get better results. Always explain your reasoning for tool selection and provide comprehensive answers."
        "Also making sure to always double check the results you get from the tools and if posible go to external web search if results"
        "are not up to expectations."
    )
)

In [17]:
# Defining helper to print messages
def print_messages(messages: List[BaseMessage]):
    for m in messages: 
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

In [18]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

run_1 = agentic_rag.invoke(
    query="When Pokémon Gold and Silver was released?", 
    session_id="ignacio",
)

print("\nMessages from run 1:")
messages = run_1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = You are an Agentic RAG assistant that can intelligently decide which tools to use to answer user questions. Reason about about the response, change the query and call the tool again if needed in order to get better results. Always explain your reasoning for tool selection and provide comprehensive answers.Also making sure to always double check the results you get from the tools and if posible go to external web search if resultsare not up to expectations., tool_calls = None)
 -> (role =

In [19]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

run_2 = agentic_rag.invoke(
    query="Which one was the first 3D platformer Mario game?", 
    session_id="ignacio",
)

print("\nMessages from run 2:")
messages = run_2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = You are an Agentic RAG assistant that can intelligently decide which tools to use to answer user questions. Reason about about the response, change the query and call the tool again if needed in order to get better results. Always explain your reasoning for tool selection and provide comprehensive answers.Also making sure to always double check the results you get from the tools and if posible go to external web search if resultsare not up to expectations., tool_calls = None)
 -> (role =

In [20]:
# TODO: Invoke your agent
# - When Pokémon Gold and Silver was released?
# - Which one was the first 3D platformer Mario game?
# - Was Mortal Kombat X realeased for Playstation 5?

run_3 = agentic_rag.invoke(
    query="Was Mortal Kombat X realeased for Playstation 5?", 
    session_id="ignacio",
)

print("\nMessages from run 3:")
messages = run_3.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 3:
 -> (role = system, content = You are an Agentic RAG assistant that can intelligently decide which tools to use to answer user questions. Reason about about the response, change the query and call the tool again if needed in order to get better results. Always explain your reasoning for tool selection and provide comprehensive answers.Also making sure to always double check the results you get from the tools and if posible go to